Trains a simple convnet on the MNIST dataset.

In [1]:
import numpy as np

In [2]:
from kerasy.datasets import mnist
from kerasy.models import Sequential
from kerasy.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input
from kerasy.utils import CategoricalEncoder

In [3]:
batch_size = 128
num_classes = 10
epochs = 12
n_samples = 1000

In [4]:
# input image dimensions
img_rows, img_cols = 28, 28

In [5]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
x_train = np.expand_dims(x_train, axis=-1)
x_test  = np.expand_dims(x_test,  axis=-1)
input_shape = (img_rows, img_cols, 1)

In [7]:
x_train = x_train[:n_samples]
y_train = y_train[:n_samples]
x_test = x_test[:n_samples]
y_test = y_test[:n_samples]

In [8]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (1000, 28, 28, 1)
1000 train samples
1000 test samples


In [9]:
# convert class vectors to binary class matrices
encoder = CategoricalEncoder()
y_train = encoder.to_onehot(y_train, num_classes)
# y_test  = encoder.to_onehot(y_test, num_classes)

In [10]:
keep_prob = 0.5

In [11]:
model = Sequential()
model.add(Input(input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(keep_prob = keep_prob))
model.add(Dense(num_classes, activation='softmax'))

In [12]:
model.compile(optimizer='sgd', loss="categorical_crossentropy")

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

 Epoch 01/12 | 1000/1000 [####################] 100.00% - 1413.128s  
 Epoch 02/12 | 1000/1000 [####################] 100.00% - 1411.117s  
 Epoch 03/12 | 1000/1000 [####################] 100.00% - 1415.613s  
 Epoch 04/12 | 1000/1000 [####################] 100.00% - 1417.975s  
 Epoch 05/12 | 1000/1000 [####################] 100.00% - 1423.850s  
 Epoch 06/12 | 1000/1000 [####################] 100.00% - 1417.951s  
 Epoch 07/12 | 1000/1000 [####################] 100.00% - 1416.087s  
 Epoch 08/12 | 0970/1000 [###################-]  97.00% - 1686.658s  

In [ ]:
print(f"Accuracy: {np.mean(y_test==np.argmax(model.predict(x_test))):.3f}%")